# Grid World example

In this notebook we implement the GridWorld game with the `environment-framework` and use stable-baseliens3 to train a `DQN`-agent on it.

In [ ]:
from enum import Enum
from random import randint
from typing import Any, List, Tuple

from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.dqn import DQN

from numpy.typing import NDArray
from gymnasium.spaces import Space, Discrete

import cv2
import numpy as np

from gymnasium.spaces import Space, Box
import math
import numpy as np

from environment_framework import Level
from environment_framework import EnvironmentFrameworkGym
from environment_framework import Simulator

## Implement the `Game`

In [ ]:
class Action(Enum):
    UP = 0
    DOWN = 1
    RIGHT = 2
    LEFT = 3


class GridWorldGame:
    def __init__(self, size: int) -> None:
        self.size = size
        self.player_position = (0, 0)
        self.target_position = (0, 0)
        self.reset()

    @property
    def done(self) -> bool:
        return self.player_position == self.target_position
    
    @property
    def space(self) -> Space:
        return Discrete(4)
        

    def act(self, action: Action, **_: Any) -> None:
        if action == Action.UP:
            self.player_position = (self.player_position[0], self.player_position[1] - 1)
        if action == Action.DOWN:
            self.player_position = (self.player_position[0], self.player_position[1] + 1)
        if action == Action.RIGHT:
            self.player_position = (self.player_position[0] + 1, self.player_position[1])
        if action == Action.LEFT:
            self.player_position = (self.player_position[0] - 1, self.player_position[1])
        corrected_x = max(0, min(self.size - 1, self.player_position[0]))
        corrected_y = max(0, min(self.size - 1, self.player_position[1]))
        self.player_position = (corrected_x, corrected_y)

    def reset(self) -> None:
        def get_random_position() -> int:
            return randint(0, self.size - 1)

        self.player_position = (get_random_position(), get_random_position())
        self.target_position = (get_random_position(), get_random_position())
        if self.done:
            self.reset()

## Implement the `Observer` and the `Estimator`

In [ ]:
class GridWorldObserver:
    def __init__(self, game: GridWorldGame) -> None:
        self.game = game

    @property
    def space(self) -> Space:
        return Box(shape=(4,), low=-math.inf, high=math.inf)

    def observe(self, _: Any) -> NDArray:
        return np.array([*self.game.player_position, *self.game.target_position])


class GridWorldEstimator:
    def __init__(self, game: GridWorldGame) -> None:
        self.game = game

    def estimate(self, _: Any) -> float:
        return -1 + float(self.game.done)

## Add a nice little `Visualizer`

In [ ]:
class GridWorldVisualizer:
    # We use BGR
    BLUE = [255, 0, 0]
    GREEN = [0, 255, 0]

    def __init__(self, game: GridWorldGame) -> None:
        self.game = game

    def render(self, _: Any) -> Any:
        frame = [[[0 for k in range(3)] for j in range(self.game.size)] for i in range(self.game.size)]
        frame[self.game.player_position[1]][self.game.player_position[0]] = self.BLUE
        frame[self.game.target_position[1]][self.game.target_position[0]] = self.GREEN
        return frame

## Connect all together with a `Level`

In [ ]:
class GridWorldLevel(Level):
    _game: GridWorldGame
    _observer: GridWorldObserver
    _estimator: GridWorldEstimator
    _visualizer: GridWorldVisualizer

    def __init__(
        self,
        game: GridWorldGame,
        observer: GridWorldObserver,
        estimator: GridWorldEstimator,
        visualizer: GridWorldVisualizer,
    ) -> None:
        super().__init__(game, observer, estimator, visualizer)

    def reset(self) -> None:
        self._game.reset()

    def step(self, action: Action) -> Any:
        if isinstance(action, np.int64):  # handle integer inputs
            action = Action(action)
        self._game.act(action)

## Look at a random selecting agent

In [ ]:
game = GridWorldGame(7)
scale_factor = 50
level = GridWorldLevel(game, GridWorldObserver(game), GridWorldEstimator(game), GridWorldVisualizer(game))
simulator = Simulator(level)
while not simulator.done:
    action = Action(randint(0, 3))
    simulator.step(action)
    frame = np.array(simulator.render(), dtype=np.uint8)
    frame = cv2.resize(frame, (scale_factor * game.size, scale_factor * game.size), interpolation=cv2.INTER_AREA)
    cv2.imshow("GridWorld", frame)
    cv2.waitKey(33)
cv2.waitKey(500)
cv2.destroyAllWindows()

## Use stable-baselines3 to train an DQN-agent in the environment

In [ ]:
game = GridWorldGame(7)
level = GridWorldLevel(game, GridWorldObserver(game), GridWorldEstimator(game), GridWorldVisualizer(game))
env = EnvironmentFrameworkGym(level, render_mode="rgb_array")

model = DQN("MlpPolicy", env)
model.learn(
    total_timesteps=int(5e5),
    progress_bar=True,
)

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10)

In [ ]:
model.save("gridworld-dqn.zip")

In [ ]:
game = GridWorldGame(7)
scale_factor = 50
level = GridWorldLevel(game, GridWorldObserver(game), GridWorldEstimator(game), GridWorldVisualizer(game))
env = EnvironmentFrameworkGym(level, render_mode="rgb_array")

model = DQN.load("gridworld-dqn.zip", env=env)
vec_env = model.get_env()
obs = vec_env.reset()
for _ in range(50):
    obs = np.array(obs)
    action, _states = model.predict(obs)  # type: ignore
    obs, _, _, _ = vec_env.step(action)
    frame = np.array(vec_env.render(), dtype=np.uint8)
    frame = cv2.resize(frame, (scale_factor * game.size, scale_factor * game.size), interpolation=cv2.INTER_AREA)
    cv2.imshow("GridWorld", frame)
    cv2.waitKey(250)
cv2.waitKey(500)
cv2.destroyAllWindows()